In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Reading the Dataset
dataset = pd.read_csv('insurance_pre.csv')
#its a nominal value so values should be expanded
dataset=pd.get_dummies(dataset,drop_first=True)
dataset=dataset.astype(int) # expanded columns are boolean so converted into int
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [3]:
#split input and output
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dependent=dataset[['charges']]
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)
X_test.shape#to check number of rows and columns

(446, 5)

In [ ]:
!pip install lightgbm

In [5]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
X_train

array([[-1.53963418,  0.13333747, -0.90788827, -0.98885138, -0.49929923],
       [ 0.74809711,  0.29449875, -0.0755796 ,  1.01127431,  2.00280702],
       [ 0.4621307 ,  2.06727281,  1.58903774,  1.01127431, -0.49929923],
       ...,
       [ 0.03318108, -0.8336302 , -0.90788827,  1.01127431, -0.49929923],
       [-1.46814257,  0.77798258, -0.90788827,  1.01127431, -0.49929923],
       [-0.46726014, -1.96175915, -0.0755796 , -0.98885138, -0.49929923]])

In [7]:
from sklearn.preprocessing import StandardScaler
scy=StandardScaler()
y_train=scy.fit_transform(y_train)
y_test=scy.transform(y_test)
y_train

array([[-9.74679846e-01],
       [ 2.43598702e+00],
       [-3.54958705e-01],
       [-7.40316579e-01],
       [-7.72986937e-01],
       [-3.56143640e-01],
       [-3.12385674e-01],
       [-4.57624882e-01],
       [-1.32190877e-01],
       [-7.93977219e-01],
       [-9.29652306e-01],
       [-8.53308620e-01],
       [-7.77134210e-01],
       [-4.74975719e-01],
       [ 1.80289304e+00],
       [-1.77599858e-02],
       [-5.06630418e-01],
       [ 1.10860562e+00],
       [-1.18056292e-01],
       [-9.62238025e-01],
       [-1.95754189e-01],
       [-1.98208698e-01],
       [-7.48357211e-01],
       [-9.73833463e-01],
       [ 2.62921611e+00],
       [-4.66765811e-01],
       [ 1.28668446e+00],
       [-8.07773251e-01],
       [ 9.62773941e-01],
       [-6.47468438e-01],
       [-1.09677107e-01],
       [ 3.90196296e-01],
       [ 1.75727303e+00],
       [-3.02313724e-01],
       [-4.33418347e-01],
       [-9.67147043e-01],
       [-7.53181590e-01],
       [-4.09296451e-01],
       [-9.3

In [9]:
from lightgbm import LGBMRegressor
regressor = LGBMRegressor(
  n_estimators=50,      # Number of boosting rounds
    max_depth=3   # Unlimited depth (default)
)
regressor.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000512 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 95
[LightGBM] [Info] Number of data points in the train set: 892, number of used features: 5
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

C:\Users\priyanka\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMRegressor(max_depth=3, n_estimators=50)

In [11]:
#get predicted output 
y_pred=regressor.predict(X_test)
#Validation
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.8933181550838848

In [13]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(regressor, open(filename, 'wb'))
pickle.dump(sc, open('scaler.sav', 'wb'))
pickle.dump(scy, open('scalery.sav', 'wb'))

In [31]:
loaded_model = pickle.load(open('finalized_model.sav', 'rb'))
loaded_sc = pickle.load(open('scaler.sav', 'rb'))
loaded_scy = pickle.load(open('scalery.sav', 'rb'))

In [33]:
preproc= loaded_sc.transform([[45,34,2,1,0]])

C:\Users\priyanka\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [35]:
preproc

array([[ 0.39063909,  0.61682131,  0.75672907,  1.01127431, -0.49929923]])

In [37]:
result = loaded_model.predict(preproc)
preprocResult=loaded_scy.inverse_transform(result.reshape(-1, 1))
print(preprocResult)

[[10335.41394902]]
